In [4]:
# Syntax für das SAT-Problem
# Alle Funktionen produzieren Strings in SMTLib-Syntax, die direkt mit gängigen SAT-Lösern verwendet können,
# z.B. mit dem Web-Interface für Z3 unter https://compsys-tools.ens-lyon.fr/z3/index.php

# eine propositionale Formel (produziert mit den Funktionen unten) als Input für SAT
def SATInput(variablen, formel):
    return "".join(["(declare-fun " + v + " () Bool)\n" for v in variablen]) + "(assert " + formel + ")\n" + "(check-sat)(get-model)"

# Konjunktion einer Liste von Formeln
def AND(formeln):
    if formeln == []:
        return "true"
    else:
        return "(and " + " ".join(formeln) + ")"

# Disjunktion einer Liste von Formeln
def OR(formeln):
    if formeln == []:
        return "false"
    else:
        return "(or " + " ".join(formeln) + ")"

# Negation einer Formel
def NOT(formel):
    return "(not " + formel + ")"

# Implikation formel1 => formel2
def IMPL(formel1,formel2):
    return "(=> " + formel1 + " " + formel2 + ")"


In [5]:
# reduziert HamiltonPath für Graph G auf SAT
# Input: Graph G gegeben als Paar aus den Listen von Knoten und Kanten (k,l)
# Ouptut: propositionale Formel als String in SMTLib-Syntax
def HS(G):
    # Liste der Knoten und Liste der Kanten
    (Kno,Kan) = G
    # die Anzahl der Knoten
    N = len(Kno)
    # die Liste der Positionen im Pfad
    Positionen = range(1,N+1)
    # successors[k] ist die Liste der k', so dass G die Kante (k,l) hat
    succ = {}
    for k in Kno:
        succ[k] = []
    for (k,l) in Kan:
         succ[k].append(l)
            
    def Pos(k,p):
        return "pos_" + str(k) + "_" + str(p)
    
    # die Liste der propositionalen Variablen
    variablen = [Pos(k,p) for k in Kno for p in Positionen]
    
    # Bauen der Teil-Formeln
    
    jedePositionHatEinenKnoten = AND([OR([Pos(k,p) for k in Kno]) for p in Positionen])
    
    # nicht k und l an Position p
    def nichtInGleicherPosition(k,l,p):
        return NOT(AND([Pos(k,p), Pos(l,p)]))
    keinePositionHatZweiKnoten = AND([nichtInGleicherPosition(k,l,p) for p in Positionen for k in Kno for l in Kno if k != l])
    
    jederKnotenIstEinmalImPfad = AND([OR([Pos(k,p) for p in Positionen]) for k in Kno])
    
    # nicht k an Positionen p und q
    def nichtAnZweiPositionen(k,p,q):
        return NOT(AND([Pos(k,p), Pos(k,q)]))
    keinKnotenIstZweimalImPfad = AND([nichtAnZweiPositionen(k,p,q) for k in Kno for p in Positionen for q in Positionen if p != q])
    
    # wenn k an Position p, dann successor von k an Position p+1
    def folgePositionHatNachfolger(k,p):
        return IMPL(Pos(k,p), OR([Pos(l,p+1) for l in succ[k]]))
    alleFolgePositionenHabenNachfolger = AND([folgePositionHatNachfolger(k,p) for k in Kno for p in Positionen[:-1]])
    
    # HS(G): Konjunktion aller Teilformeln
    isHamiltonPfad = AND([jedePositionHatEinenKnoten, keinePositionHatZweiKnoten, jederKnotenIstEinmalImPfad, keinKnotenIstZweimalImPfad, alleFolgePositionenHabenNachfolger])
    
    # Formatierung als SMTLIB-Problem
    return SATInput(variablen, isHamiltonPfad)

print(HS((["A","B","C"], [("A","B"), ("A","C"), ("B","C")])))

(declare-fun pos_A_1 () Bool)
(declare-fun pos_A_2 () Bool)
(declare-fun pos_A_3 () Bool)
(declare-fun pos_B_1 () Bool)
(declare-fun pos_B_2 () Bool)
(declare-fun pos_B_3 () Bool)
(declare-fun pos_C_1 () Bool)
(declare-fun pos_C_2 () Bool)
(declare-fun pos_C_3 () Bool)
(assert (and (and (or pos_A_1 pos_B_1 pos_C_1) (or pos_A_2 pos_B_2 pos_C_2) (or pos_A_3 pos_B_3 pos_C_3)) (and (not (and pos_A_1 pos_B_1)) (not (and pos_A_1 pos_C_1)) (not (and pos_B_1 pos_A_1)) (not (and pos_B_1 pos_C_1)) (not (and pos_C_1 pos_A_1)) (not (and pos_C_1 pos_B_1)) (not (and pos_A_2 pos_B_2)) (not (and pos_A_2 pos_C_2)) (not (and pos_B_2 pos_A_2)) (not (and pos_B_2 pos_C_2)) (not (and pos_C_2 pos_A_2)) (not (and pos_C_2 pos_B_2)) (not (and pos_A_3 pos_B_3)) (not (and pos_A_3 pos_C_3)) (not (and pos_B_3 pos_A_3)) (not (and pos_B_3 pos_C_3)) (not (and pos_C_3 pos_A_3)) (not (and pos_C_3 pos_B_3))) (and (or pos_A_1 pos_A_2 pos_A_3) (or pos_B_1 pos_B_2 pos_B_3) (or pos_C_1 pos_C_2 pos_C_3)) (and (not (and pos_A_